# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df_csv = "weather_df.csv"
weather_df2 = pd.read_csv(weather_df_csv)
weather_df2.head()

,City,Country,Latitude,Longitude,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed,Date
0,Kavieng,PG,-2.57,150.80,82.40,75,92,10.85,1602840999
1,Waynesville,US,35.49,-82.99,51.01,84,88,5.03,1602840999
2,Bandarbeyla,SO,9.49,50.81,88.16,52,0,19.01,1602840999
3,Tilichiki,RU,60.47,166.10,32.02,93,97,11.34,1602840999
4,Presidencia Roque Sáenz Peña,AR,-26.79,-60.44,63.86,77,72,3.18,1602840967


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = weather_df2[["Latitude", "Longitude"]]
humidity = weather_df2["Humidity (%)"]
max_intensity = max(weather_df2["Humidity (%)"])

fig = gmaps.figure()

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout) 

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_intensity,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
revised_df = weather_df2[weather_df2["Max Temperature"] < 80]
revised_df = revised_df[revised_df["Max Temperature"] > 70]
revised_df = revised_df[revised_df["Wind Speed"] < 10]
revised_df = revised_df[revised_df["Cloudiness (%)"] == 0]
revised_df
#drop any null values

,City,Country,Latitude,Longitude,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed,Date
78,Tunceli,TR,39.11,39.55,77.00,20,0,1.12,1602840714
89,Gilgit,PK,35.92,74.31,73.33,18,0,1.32,1602841011
106,San Quintín,MX,30.48,-115.95,70.11,56,0,5.26,1602841013
196,Lüderitz,NaN,-26.65,15.16,71.26,47,0,7.96,1602840750
288,Bulancak,TR,40.94,38.23,73.40,78,0,4.70,1602841040
308,Kaka,TM,37.35,59.61,74.62,18,0,6.51,1602841042
344,Bundaberg,AU,-24.85,152.35,71.60,68,0,8.05,1602841047
353,Petra,JO,30.32,35.48,78.06,30,0,3.71,1602841049
467,Amapá,BR,1.00,-52.00,70.65,90,0,1.25,1602841065
501,Gumdag,TM,39.21,54.59,78.76,15,0,9.78,1602840786


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = revised_df
hotel_df["Hotel Name"] = ""

In [7]:
params = {
    "location": "",
    "keyword": "hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    params["location"] = f"{latitude},{longitude}"
    

    hotels = requests.get(base_url, params).json()
    
    try:
        hotel_name = hotels["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name  
        print(f"The first hotel in '{row['City']}' is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for '{row['City']}'... skipping.")   
        print("-------------------------------------------------------------------------")

The first hotel in 'Tunceli' is Grand Hotel Şaroğlu
-------------------------------------------------------------------------
The first hotel in 'Gilgit' is Canopy Nexus GILGIT
-------------------------------------------------------------------------
The first hotel in 'San Quintín' is Old Mill Hotel
-------------------------------------------------------------------------
The first hotel in 'Lüderitz' is LÜDERITZ NEST HOTEL
-------------------------------------------------------------------------
The first hotel in 'Bulancak' is ELİT OTEL BULANCAK
-------------------------------------------------------------------------
Could not find information for 'Kaka'... skipping.
-------------------------------------------------------------------------
The first hotel in 'Bundaberg' is Burnett Riverside Hotel
-------------------------------------------------------------------------
The first hotel in 'Petra' is Petra Marriott Hotel
---------------------------------------------------------------

In [8]:
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed,Date,Hotel Name
78,Tunceli,TR,39.11,39.55,77.00,20,0,1.12,1602840714,Grand Hotel Şaroğlu
89,Gilgit,PK,35.92,74.31,73.33,18,0,1.32,1602841011,Canopy Nexus GILGIT
106,San Quintín,MX,30.48,-115.95,70.11,56,0,5.26,1602841013,Old Mill Hotel
196,Lüderitz,NaN,-26.65,15.16,71.26,47,0,7.96,1602840750,LÜDERITZ NEST HOTEL
288,Bulancak,TR,40.94,38.23,73.40,78,0,4.70,1602841040,ELİT OTEL BULANCAK
308,Kaka,TM,37.35,59.61,74.62,18,0,6.51,1602841042,
344,Bundaberg,AU,-24.85,152.35,71.60,68,0,8.05,1602841047,Burnett Riverside Hotel
353,Petra,JO,30.32,35.48,78.06,30,0,3.71,1602841049,Petra Marriott Hotel
467,Amapá,BR,1.00,-52.00,70.65,90,0,1.25,1602841065,
501,Gumdag,TM,39.21,54.59,78.76,15,0,9.78,1602840786,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…